# Testing with making the bubble plot

In [1]:
import pandas as pd
import matplotlib.image as mpimg
import plotly.express as px
import plotly.graph_objects as go
from matplotlib import image, pyplot
import matplotlib.image as mpimg
import  PIL.Image as Image


In [14]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

class model_trainer():
    def __init__(self, name):
        self.car_type = "2P"

        self.df = pd.read_csv("..\..\data\MC1\SensorDataProcessed.csv")

    def run_prediction(self):
        # read df from csv
        df_x = pd.read_csv("..\..\data\MC1\one_hot_encoding.csv")
        y_values = self.get_y_values(self.car_type)
        x_train, x_test, y_train, y_test = train_test_split(df_x, y_values, test_size=0.20, random_state=69)

        logistic_model = LogisticRegression(random_state=0).fit(x_train, y_train)
        # return the coefficients of the model
        return logistic_model.coef_[0]
    
    def get_y_values(self, car_type):
        
        def is_car_type(type):
            return "1" if type == car_type else "0"

        cars = self.df.groupby(by="car-id").first()
        encoded_cars = cars.replace({"car-type": {"1": is_car_type("1"),
                                                  "2": is_car_type("2"),
                                                  "2P": is_car_type("2P"),
                                                  "3": is_car_type("3"),
                                                  "4": is_car_type("4"),
                                                  "5": is_car_type("5"),
                                                  "6": is_car_type("6")}})

        return encoded_cars.sort_values(by="Timestamp")['car-type']

In [15]:
trainer = model_trainer('test')
coefficients = trainer.run_prediction()

In [16]:
coefficients

array([-1.48427207e+00, -5.56521416e-01, -2.24259512e-01,  1.00721577e-01,
        9.04149562e-02,  3.96565470e-01, -1.52128059e+00, -1.34959336e+00,
       -9.77255956e-02, -3.13634902e-01, -1.38376189e+00, -7.51369135e-02,
       -4.08299289e-02, -1.21783123e-01,  4.10708678e+00,  4.10708678e+00,
       -9.64999379e-02,  4.21451575e-01,  4.21451575e-01, -4.58374113e-01,
        2.01844998e-01, -2.73384687e-01,  6.24112521e-01, -2.72555327e-01,
        6.24112521e-01,  6.34038058e-01,  6.34038058e-01, -1.42934108e+00,
        4.02087492e-01,  1.40699120e+00,  2.54097106e-01,  3.83027752e-01,
        3.83027752e-01, -1.16425713e-01, -2.50242331e-01, -2.55638244e-01,
       -1.23846570e-03, -1.74595634e-02,  3.82792822e-01,  3.82792822e-01])

In [17]:
locations = pd.read_parquet("..\..\data\MC1\locations.parquet")
coordinates = locations['coordinates']
img = mpimg.imread("..\..\data\MC1\Lekagul Roadways.bmp")
image = Image.open("..\..\data\MC1\Lekagul Roadways.bmp")

In [37]:
def make_car_bubbles(coordinates, coefficients, img)->go.Figure:
    # get width and height of img PIL
    img_width, img_height = image.size
    scale_factor = 3

    coefficients = [abs(x) for x in coefficients]

    x_values = [x[0]*scale_factor for x in coordinates]
    y_values = [x[1]*scale_factor for x in coordinates]

    print(x_values)
    print(y_values)

    df_plot = pd.DataFrame({'x': x_values, 'y': y_values, 'size': coefficients})

    fig = px.scatter(df_plot, x='x', y='y', size='size')

    fig.update_xaxes(
        visible=False,
    )

    fig.update_yaxes(
        visible=False,
    )

    # Add image
    fig.add_layout_image(
        dict(
            x=0,
            sizex=img_width * scale_factor,
            y=img_height * scale_factor,
            sizey=img_height * scale_factor,
            xref="x",
            yref="y",
            opacity=1.0,
            layer="below",
            sizing="stretch",
            source=img)
    )

    # Configure other layout
    fig.update_layout(
        width=img_width * scale_factor,
        height=img_height * scale_factor,
        margin={"l": 0, "r": 0, "t": 0, "b": 0},
    )

    return fig

In [38]:
fig = make_car_bubbles(coordinates, coefficients, image)

[345, 192, 240, 267, 312, 372, 420, 51, 195, 207, 549, 450, 132, 135, 384, 414, 156, 447, 444, 549, 558, 408, 492, 543, 453, 393, 369, 186, 60, 54, 348, 291, 300, 144, 72, 57, 330, 387, 174, 189]
[99, 525, 495, 552, 504, 267, 48, 399, 168, 306, 339, 69, 408, 396, 75, 57, 477, 420, 465, 456, 435, 189, 258, 165, 246, 162, 159, 561, 237, 315, 147, 120, 144, 333, 438, 528, 573, 450, 468, 501]


In [39]:
px.imshow(img)

In [40]:
locations

,location,coordinates
0,entrance3,"[115, 33]"
1,general-gate1,"[64, 175]"
2,ranger-stop2,"[80, 165]"
3,ranger-stop0,"[89, 184]"
4,general-gate2,"[104, 168]"
5,general-gate5,"[124, 89]"
6,entrance4,"[140, 16]"
7,entrance1,"[17, 133]"
8,general-gate7,"[65, 56]"
9,general-gate4,"[69, 102]"


In [41]:
coefficients

array([-1.48427207e+00, -5.56521416e-01, -2.24259512e-01,  1.00721577e-01,
        9.04149562e-02,  3.96565470e-01, -1.52128059e+00, -1.34959336e+00,
       -9.77255956e-02, -3.13634902e-01, -1.38376189e+00, -7.51369135e-02,
       -4.08299289e-02, -1.21783123e-01,  4.10708678e+00,  4.10708678e+00,
       -9.64999379e-02,  4.21451575e-01,  4.21451575e-01, -4.58374113e-01,
        2.01844998e-01, -2.73384687e-01,  6.24112521e-01, -2.72555327e-01,
        6.24112521e-01,  6.34038058e-01,  6.34038058e-01, -1.42934108e+00,
        4.02087492e-01,  1.40699120e+00,  2.54097106e-01,  3.83027752e-01,
        3.83027752e-01, -1.16425713e-01, -2.50242331e-01, -2.55638244e-01,
       -1.23846570e-03, -1.74595634e-02,  3.82792822e-01,  3.82792822e-01])

In [42]:
fig.show()